## TF-IDF

In [ ]:
from pprint import pprint
import gensim

# 提取指定门类进行 TF-IDF 矩阵构建：
for N in [8,10,5,4,12,9,7,2,13,3,1]:
    result = []
    for i in range(cut_word_list.shape[0]):
        if cut_word_list.iloc[i]['Tag'] == int(N):
            result.append(cut_word_list.iloc[i]['分词结果'])
    result = pd.DataFrame(result,columns=['分词结果'])
    
     # 计算 TF-IDF 值
    tfvectorizer=CountVectorizer(lowercase=False) 
    count_vector=tfvectorizer.fit_transform(result['分词结果']) # Tf 矩阵
    transformer = TfidfTransformer() # 转换 Tf 矩阵
    tfidf = transformer.fit_transform(count_vector) # 将 TF 转换成 Tf-Idf
    tfvectorizer_feature_names = tfvectorizer.get_feature_names()
    tfvectorizer_vocabulary = tfvectorizer.vocabulary_
    tfidf_result = tfidf.toarray()
    tfidf.toarray().sum(axis=0)
    data = {'word': tfvectorizer_feature_names,
        'tfidf': tfidf.toarray().sum(axis=0).tolist()}
    key_word = pd.DataFrame(data)
    key_word_sort=key_word.sort_values(by="tfidf" , ascending=False) 
    final_key=key_word_sort.head(100)
    
    
    for j in range(1,5):
        simi_dimension=[]
        standard = pd.read_excel('standard'+str(j) +'10.xlsx')
        standard_word=standard["word"]
        standard_len=len(standard_word)
        comment_word=final_key["word"]
        commet_len=len(comment_word)
        all_word=[]
        for s in standard_word:
            all_word.append(s)
        for m in comment_word:
            all_word.append(m)
   
        model =gensim.models.Word2Vec([all_word],min_count=0,window=5)
        for a in range(commet_len):
            all_similarity=[]
            for b in range(standard_len):
                similarity=model.wv.similarity(all_word[standard_len+a],all_word[b])
                all_similarity.append(similarity)
    
            simi_dimension.append(max(all_similarity))
        final_key["similarity{}".format(j)]=simi_dimension  
     
    #获取每个类别各个维度的词典
    #final_key.to_excel('dictionary'+str(N) +'.xlsx' , index=None)
    dimesion1=[]
    dimesion2=[]
    dimesion3=[]
    dimesion4=[]
    final_simi=final_key[["similarity1","similarity2","similarity3","similarity4",]]
    index_simi=final_simi.idxmax(axis=1)
    for i in range(len(index_simi)):
        if index_simi.iloc[i]=="similarity1":
            dimesion1.append(final_key.iloc[i,0])
        elif index_simi.iloc[i]=="similarity2":
            dimesion2.append(final_key.iloc[i,0])
        elif index_simi.iloc[i]=="similarity3":
            dimesion3.append(final_key.iloc[i,0])
        else:
            dimesion4.append(final_key.iloc[i,0])
            
    df_dime1 = pd.DataFrame(dimesion1, columns=['选题与综述'])
    df_dime2 = pd.DataFrame(dimesion2, columns=['创新与论文价值'])
    df_dime3 = pd.DataFrame(dimesion3, columns=['科研能力与基础知识'])
    df_dime4 = pd.DataFrame(dimesion4, columns=['论文规范性'])
    dim_word = pd.concat([df_dime1,df_dime2,df_dime3,df_dime4],axis=1)
    dim_word.to_excel('final词典'+str(N) +'.xlsx' , index=None)
    #tfidf_result = pd.DataFrame(tfidf_result,columns=tfvectorizer_feature_names)
    #tfidf_result = pd.DataFrame(tfidf_result,columns=tfvectorizer_feature_names)
    #print(N)
    #paths = 'subject'+str(N) +'tf_idf.xlsx' 
    #tfidf_result.to_excel(paths,encoding = 'utf-8-sig')